In [1]:
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Activation, LSTM, Flatten, Reshape
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.python.keras import backend as K
import gc
import tensorflow as tf
import random
from sklearn import preprocessing
import pickle

In [2]:
#Load config
with open('Data_Set/config.pickle', 'rb') as data:
    config = pickle.load(data)
dataSetSize = config["dataSetSize"]
testSetBenignSize = config["testSetBenignSize"]
validationSetSize = config["validationSetSize"]
trainingSetSize = config["trainingSetSize"]
sequenceLen = config["sequenceLen"]
dimensionsCount = config["dimensionsCount"]
numberOfAttackSamplesToChoose = 100
testSetSize = testSetBenignSize + numberOfAttackSamplesToChoose

In [3]:
with open('Data_set/normalized_data_set.pickle', 'rb') as data:
    normalizedDataSet = pickle.load(data)
with open('Data_set/normalized_angle_shift_attack_data_set.pickle', 'rb') as data:
    normalizedAngleShiftAttackDataSet = pickle.load(data)

testSetBenign = np.reshape(normalizedDataSet[trainingSetSize:trainingSetSize+testSetBenignSize,:,:,:], (testSetBenignSize,sequenceLen,dimensionsCount), order = 'C')
testSetAttack = np.reshape(normalizedAngleShiftAttackDataSet[0:numberOfAttackSamplesToChoose,:,:,:], (numberOfAttackSamplesToChoose,sequenceLen,dimensionsCount), order = 'C')
testSet = np.concatenate((testSetBenign,testSetAttack))
test_labels = np.zeros(testSetSize)
test_labels[testSetBenignSize:] = np.ones(numberOfAttackSamplesToChoose) 

In [4]:
testSetSize

445

In [5]:
def rankedPrecision(mse_labels):
    sorted_mse_label = mse_label[mse_label[:,0].argsort()[::-1]]
    totalNumberOfAnomalies = sum(sorted_mse_label[:,1] == 1)
    TP = sum(sorted_mse_label[0:totalNumberOfAnomalies,1] == 1)
    FP = sum(sorted_mse_label[0:totalNumberOfAnomalies,1] == 0)
    TN = sum(sorted_mse_label[totalNumberOfAnomalies:,1] == 0)
    FN = sum(sorted_mse_label[totalNumberOfAnomalies:,1] == 1)
    precision = TP/(TP+FP)
    return precision
    
    
def rankedRecall(mse_labels):
    sorted_mse_label = mse_label[mse_label[:,0].argsort()[::-1]]
    totalNumberOfAnomalies = sum(sorted_mse_label[:,1] == 1)
    TP = sum(sorted_mse_label[0:totalNumberOfAnomalies,1] == 1)
    FP = sum(sorted_mse_label[0:totalNumberOfAnomalies,1] == 0)
    TN = sum(sorted_mse_label[totalNumberOfAnomalies:,1] == 0)
    FN = sum(sorted_mse_label[totalNumberOfAnomalies:,1] == 1)
    recall = TP/(TP+FN)
    return recall

# show_curve() is for plotting training and validation losses
def show_curve(history):
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'validation'], loc='upper left')
    plt.show()
    
    
def rankedPrecisionThre(mse_labels,threshold):
    sorted_mse_label = mse_label[mse_label[:,0].argsort()[::-1]]
    TP = sum((sorted_mse_label[:,1] == 1) & (sorted_mse_label[:,0] >= threshold))
    FP = sum((sorted_mse_label[:,1] == 0) & (sorted_mse_label[:,0] >= threshold))
    TN = sum((sorted_mse_label[:,1] == 0) & (sorted_mse_label[:,0] < threshold))
    FN = sum((sorted_mse_label[:,1] == 1) & (sorted_mse_label[:,0] < threshold))
    precision = TP/(TP+FP)
    return precision
    
    
def rankedRecallThre(mse_labels,threshold):
    sorted_mse_label = mse_label[mse_label[:,0].argsort()[::-1]]
    TP = sum((sorted_mse_label[:,1] == 1) & (sorted_mse_label[:,0] >= threshold))
    FP = sum((sorted_mse_label[:,1] == 0) & (sorted_mse_label[:,0] >= threshold))
    TN = sum((sorted_mse_label[:,1] == 0) & (sorted_mse_label[:,0] < threshold))
    FN = sum((sorted_mse_label[:,1] == 1) & (sorted_mse_label[:,0] < threshold))
    recall = TP/(TP+FN)
    return recall

In [6]:
precisions = []
recalls = []
model = load_model('Trained_Model/lstm_adv.h5')
predicted = model.predict(testSet)
mse = (np.square(testSet - predicted)).mean(axis=2).mean(axis=1)
mse_label = np.vstack((mse, test_labels)).T
precision = rankedPrecision(mse_label)
recall = rankedRecall(mse_label)
precisions.append(precision)
recalls.append(recall)
print("Precision is: ", precision)
print("Recall is: ", precision)

LSTMPrecisions_domain = precisions

sorted_mse_label = mse_label[mse_label[:,0].argsort()[::-1]]

totalNumberOfAnomalies = sum(sorted_mse_label[:,1] == 1)
print(min(sorted_mse_label[sorted_mse_label[:,1] == 1,0])) # 0.010118610983828593
print(max(sorted_mse_label[sorted_mse_label[:,1] == 0,0])) 

Precision is:  1.0
Recall is:  1.0
0.13461270835611572
7.710547100442337e-05


In [7]:
sorted_mse_label = mse_label[mse_label[:,0].argsort()[::-1]]
minMSE = min(sorted_mse_label[sorted_mse_label[:,1] == 0,0])
maxMSE = max(sorted_mse_label[sorted_mse_label[:,1] == 0,0])
thre = maxMSE + (maxMSE - minMSE)/10
threshold = 1
#print((sorted_mse_label[:,1] == 1) & (sorted_mse_label[:,0] >= threshold))
print("Selected threshold: ",thre)
precision = rankedPrecisionThre(mse_label,thre)
recall = rankedRecallThre(mse_label,thre)
print("Precision by threshold is: ", precision)
print("Recall by threshold is: ", recall)

Selected threshold:  8.286412884426776e-05
Precision by threshold is:  1.0
Recall by threshold is:  1.0


In [45]:
mse.shape

(445,)

In [10]:
# Testing adversarial samples
with open('Data_Set/adversarial_data_set.pickle', 'rb') as data:
    advDataSet = pickle.load(data)
sample = np.reshape(advDataSet[0,:,:],(1,200,9))
predicted = model.predict(sample)
mse = (np.square(sample - predicted)).mean(axis=2).mean(axis=1)
print(mse)

[7.82487482e-05]
